# Meta-Learning for Scalable Hydrological Streamline Delineation

**Authors**: Nattapon Jaroenchai <sup>a, b</sup>, Zhaonan Wang <sup>a, b</sup>, Lawrence V. Stanislawski <sup>c</sup>, Ethan Shavers <sup>c</sup>, Shaowen Wang <sup>a, b, *</sup>  

<sup>a</sup> *Department of Geography and Geographic Information Science, University of Illinois at Urbana-Champaign, Urbana, IL, USA*  
<sup>b</sup> *CyberGIS Center for Advanced Digital and Spatial Studies, University of Illinois at Urbana-Champaign, Urbana, IL, USA*  
<sup>c</sup> *U.S. Geology Survey, Center of Excellence for Geospatial Information Science, Rolla, MO, USA*  
<sup>d</sup> *School of Geoscience and Info-Physics, Central South University, Changsha, Hunan, China*  

## Abstract

Accurate streamline network data are vital for various applications, like agriculture and sustainability. Despite advancements in machine learning, applying models across different geographies remains challenging. This paper investigates whether meta-learning techniques can enhance model performance in such scenarios. Meta-learning leverages knowledge from multiple source tasks to improve the performance of the target task, offering promise in enhancing machine learning models. By hypothesizing that each study area holds unique underlying knowledge, we aim to leverage this knowledge to improve overall model performance, thereby expanding the transferability of the final model.

#### Keywords:

Streamline Delineation, Hydrology, Convolutional Neural Networks, U-net Model, Meta-Learning, Machine Learning

# Prepare the environment

We install and download neccessary resources.

In [1]:
%pip install segmentation-models &> /dev/null
!wget https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/unet_util.py

# https://stackoverflow.com/questions/75433717/module-keras-utils-generic-utils-has-no-attribute-get-custom-objects-when-im
# open the file keras.py, change all the 'init_keras_custom_objects' to 'init_tfkeras_custom_objects'.
# the location of the keras.py is in the error message. In your case, it should be in /usr/local/lib/python3.8/dist-packages/efficientnet/
!wget https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/keras.py
!cp './keras.py' '/usr/local/lib/python3.10/dist-packages/efficientnet/keras.py'
!rm './keras.py'

--2023-08-01 22:14:32--  https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/unet_util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12771 (12K) [text/plain]
Saving to: ‘unet_util.py’

unet_util.py        100%[===================>]  12.47K  --.-KB/s    in 0s      

2023-08-01 22:14:32 (49.7 MB/s) - ‘unet_util.py’ saved [12771/12771]

--2023-08-01 22:14:32--  https://raw.githubusercontent.com/N-Jaro/segmentation_model_tutorial/main/keras.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 711 [text/plain]


In [5]:
import os
import shutil
import numpy as np
import tensorflow as tf
from keras import backend as K
import segmentation_models as sm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from unet_util import dice_coef_loss, dice_coef, jacard_coef, dice_coef_loss, Residual_CNN_block, multiplication, attention_up_and_concatenate, multiplication2, attention_up_and_concatenate2, UNET_224, evaluate_prediction_result

sm.set_framework('tf.keras')
sm.framework()

'tf.keras'

In [6]:
dim_output = 1
dim_input = 8
cities = 'nyc,dc'
save_dir = './models'
model_type = 'metacnn'
update_batch_size =16
test_num_updates = 1
threshold = 0
meta_lr = 1e-5
update_lr = 1e-5
iterations = 500
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
tf.random.set_seed(180431)
img_size=(224, 224)

# get data
x_a_train = np.random.rand(100, img_size[0], img_size[1], 8)
y_a_train = np.random.rand(100, img_size[0], img_size[1], 1)
x_b_train = np.random.rand(100, img_size[0], img_size[1], 8)
y_b_train = np.random.rand(100, img_size[0], img_size[1], 1)
dataset = [x_a_train, y_a_train, x_b_train, y_b_train]

# Prepare the folder structure

In [7]:
import os
input_data = './samples/'
model_path = './models/'
prediction_path = './predicts/'
log_path = './logs/'

# Create the folder if it does not exist
os.makedirs(input_data, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)

# Avaiable backbones for Unet architechture
# 'vgg16' 'vgg19' 'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152' 'inceptionv3'
# 'inceptionresnetv2' 'densenet121' 'densenet169' 'densenet201' 'seresnet18' 'seresnet34'
# 'seresnet50' 'seresnet101' 'seresnet152', and 'attentionUnet'
backend = 'resnet50' # ResNet50 is the best model in the TL study

# Added first Convo 8 to 3 channels layers to the random init model
name = 'maml-model-' + backend + '-' + str(np.random.randint(1000000))

logdir = log_path + name
if(os.path.isdir(logdir)):
  shutil.rmtree(logdir)
os.makedirs(logdir, exist_ok=True)

# Define the inner model / learner

Here we define the inner model which is U-net model with RestNet50 as the backbones. We also define the neccesary functions to update weights and generate prediction.

In [16]:
import numpy as np
import tensorflow as tf
import segmentation_models as sm
from tensorflow.keras.optimizers import Adam

class BaseModel:
    def __init__(self, img_size, dim_input, dim_output, filter_num, update_lr,
                    meta_lr, meta_batch_size, update_batch_size, test_num_updates):
        """ Initializes the BaseModel with given parameters.
            Must call construct_model() after initializing this class! """
        # Basic properties for the neural network
        self.img_size = img_size         # tuple representing image size
        self.dim_input = dim_input       # input dimensions
        self.channels = dim_output       # number of channels (e.g., 3 for RGB images)
        self.dim_output = dim_output     # output dimensions
        self.filter_num = filter_num     # number of filters in CNN

        # Learning rate properties
        self.update_lr = update_lr       # learning rate for the update step
        self.meta_lr = meta_lr           # learning rate for meta-learning

        # Batch sizes
        self.update_batch_size = update_batch_size  # size of the update batch
        self.test_num_updates = test_num_updates    # number of updates during testing
        self.meta_batch_size = meta_batch_size      # size of the meta batch

        # # Placeholders for input data
        self.inputa = dataset[0]    # placeholder for input data A
        self.inputb = dataset[2]     # placeholder for input data B
        self.labela = dataset[1]     # placeholder for labels of data A
        self.labelb = dataset[3]     # placeholder for labels of data B

    def update(self, loss, weights):
        # Compute gradients and perform gradient descent update
        grads = tf.gradients(loss, list(weights.values()))   # Compute gradients of the loss with respect to model's weights
        gradients = dict(zip(weights.keys(), grads))        # Associate each weight with its corresponding gradient
        new_weights = dict(
            zip(weights.keys(), [weights[key] - self.update_lr * gradients[key] for key in weights.keys()]))
        # Perform a gradient descent update using the computed gradients and update_lr
        return new_weights

    def construct_cnn(self):
        #initialize U-net model with 8 channels input
        model = sm.Unet(backend, classes=1, encoder_weights=None, input_shape=(None, None, 8))

        # Compile the model with 'Adam' optimizer (0.001 is the default learning rate) and define the loss and metrics
        model.compile(optimizer=Adam(),
                      loss=dice_coef_loss,     # dice_coef_loss is a custom loss function
                      metrics=[dice_coef, 'accuracy'])
        return model.get_weights()

    def forward_cnn(self, inp, weights):
        weights = np.array(weights)
        #initialize U-net model with 8 channels input
        model = sm.Unet(backend, classes=1, encoder_weights=None, input_shape=(None, None, 8))

        model.set_weights(weights)         # Set the model's weights to the provided ones
        cnn_outputs = model.predict(inp)   # Perform a forward pass through the model with the given input
        return cnn_outputs


# Define Meta-learning Process

The class aims to enable fast adaptation to new tasks with limited training examples (few-shot learning) through the MAML approach.

In [25]:
class MetaCNN(BaseModel):
    def __init__(self, img_size, dim_input, dim_output, filter_num, update_lr,
                    meta_lr, meta_batch_size, update_batch_size,
                    test_num_updates):
        print("Initializing MetaCNN...")
        # Calling the constructor of the parent class to initialize base properties
        BaseModel.__init__(self, img_size, dim_input, dim_output, filter_num, update_lr,
                            meta_lr, meta_batch_size, update_batch_size, test_num_updates)

    def loss_func(self, pred, label):
        # Reshaping predictions and labels to ensure compatibility
        pred = tf.reshape(pred, [-1])
        label = tf.reshape(label, [-1])
        # Calculating the mean squared error loss between predictions and labels
        return tf.reduce_mean(tf.square(pred - label))

    def construct_model(self):
        # Constructing the CNN model using the parent class method
        self.weights = weights = self.construct_cnn()

        # Number of update steps for inner loop
        num_updates = self.test_num_updates

        def task_metalearn(inp):
            """ Perform gradient descent for one task in the meta-batch. """
            inputa, inputb, labela, labelb = inp
            task_outputbs, task_lossesb = [], []
            # print(inputa.shape)
            # Forward pass on the training data from the current task
            task_outputa = self.forward(inputa, weights)
            # Calculating loss on the training data
            task_lossa = self.loss_func(task_outputa, labela)
            # Updating weights using one-step gradient descent
            fast_weights = self.update(task_lossa, weights)
            # Forward and loss computation on the validation data
            output = self.forward(inputb, fast_weights)
            task_outputbs.append(output)
            task_lossesb.append(self.loss_func(output, labelb))

            # Repeated updates for num_updates - 1 times
            for j in range(num_updates - 1):
                loss = self.loss_func(self.forward(inputa, fast_weights), labela)
                fast_weights = self.update(loss, fast_weights)
                output = self.forward(inputb, fast_weights)
                task_outputbs.append(output)
                task_lossesb.append(self.loss_func(output, labelb))

            task_output = [task_outputa, task_outputbs, task_lossa, task_lossesb]
            return task_output

        # Define the output data types
        out_dtype = [tf.float32, [tf.float32] * num_updates, tf.float32, [tf.float32] * num_updates]
        # Bundle the inputs together
        inputs = (self.inputa, self.inputb, self.labela, self.labelb)
        # print(inputs)
        # Apply task_metalearn to each task in the meta-batch
        result = tf.map_fn_v2(task_metalearn, elems=inputs, dtype=out_dtype, parallel_iterations=self.meta_batch_size)
        outputas, outputbs, lossesa, lossesb = result

        # Calculate performance and optimization metrics
        self.total_loss1 = total_loss1 = tf.reduce_sum(lossesa) / tf.to_float(self.meta_batch_size)
        self.total_losses2 = total_losses2 = [tf.reduce_sum(lossesb[j]) / tf.to_float(self.meta_batch_size) for j in range(num_updates)]
        self.total_rmse1 = tf.sqrt(lossesa)
        self.total_rmse2 = [tf.sqrt(total_losses2[j]) for j in range(num_updates)]

        # Define training operations for pretraining and meta-training
        self.outputas, self.outputbs = outputas, outputbs
        self.pretrain_op = tf.train.AdamOptimizer(self.meta_lr).minimize(total_loss1)
        self.metatrain_op = tf.train.AdamOptimizer(self.meta_lr).minimize(total_losses2[num_updates-1])

        # Define fine-tuning operation
        maml_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "model/maml")
        self.finetune_op = tf.train.AdamOptimizer(self.meta_lr).minimize(total_loss1, var_list=maml_vars)

    def forward(self, inp, weights):
        # Forward pass for CNN followed by fully connected layer with sigmoid activation
        cnn_outputs = self.forward_cnn(inp, weights)
        preds = tf.nn.sigmoid(tf.matmul(cnn_outputs, weights['fc1']) + weights['b_fc1'])
        return preds

## Define the training loop

The train function which manage the input

In [10]:
def train(model, dataset, iterations, update_batch_size, save_dir, model_type):
    for epoch in range(iterations):
        if "meta" in model_type:
            idx = np.random.choice(800, update_batch_size, replace=False)
            inputa, labela = dataset[0][idx], dataset[1][idx]
            inputb, labelb = dataset[2][idx], dataset[3][idx]

            if epoch % 100 == 0:
                model_file = save_dir + "/" + model_type + "/model_" + str(epoch)
                model.save_weights(model_file)

                # Assuming your model has a method to compute total_rmse1 and total_rmse2
                res = model.total_rmse([inputa, inputb], [labela, labelb])
                print(epoch, res)
            else:
                if "meta" in model_type:
                    # Assuming your model has a metatrain method
                    model.metatrain([inputa, inputb], [labela, labelb])
                elif "pretrain" in model_type:
                    # Assuming your model has a pretrain method
                    model.pretrain([inputa, inputb], [labela, labelb])
        else:
            raise Exception(NotImplementedError)


In [ ]:
# get model
print(model_type, "meta" in model_type)
if "meta" in model_type:
    model = MetaCNN(img_size=img_size, dim_input=dim_input,
                    dim_output=dim_output, filter_num=64,
                      update_lr=update_lr, meta_lr=meta_lr,
                      meta_batch_size=len(cities),
                      update_batch_size=update_batch_size,
                      test_num_updates=test_num_updates)
else:
    raise Exception(NotImplementedError)

model.construct_model()

print("Training:", model_type)
train(model, dataset, sess, saver)

In [26]:
model.construct_model()

(array([[[[0.51221054, 0.55227973, 0.64364149, ..., 0.33423937,
          0.21033131, 0.98164061],
         [0.67711642, 0.06322957, 0.58023241, ..., 0.26219249,
          0.49650393, 0.77497646],
         [0.05290101, 0.41855702, 0.57108505, ..., 0.169139  ,
          0.45601451, 0.75125993],
         ...,
         [0.24945011, 0.07351996, 0.8528274 , ..., 0.10988296,
          0.91268757, 0.35114516],
         [0.14338903, 0.82920817, 0.66273644, ..., 0.67610671,
          0.23703953, 0.17105115],
         [0.22641746, 0.26768677, 0.27350547, ..., 0.25981574,
          0.39504394, 0.96058992]],

        [[0.68675414, 0.93803968, 0.4391031 , ..., 0.91853897,
          0.0680918 , 0.34352109],
         [0.9349032 , 0.91258574, 0.87401087, ..., 0.15964497,
          0.05643008, 0.556484  ],
         [0.09315405, 0.6908872 , 0.51859559, ..., 0.34348272,
          0.77120162, 0.57709145],
         ...,
         [0.48537672, 0.67482573, 0.15467096, ..., 0.67363782,
          0.95896882, 0.

/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py:371: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return py_builtins.overload_of(f)(*args)


ValueError: ignored